# Crawler de Contatos dos Grupos WhatsApp

Este notebook busca todos os contatos dos grupos do WhatsApp usando a Evolution API e salva em Excel e CSV.


In [6]:
import os
import asyncio
import httpx
import pandas as pd
from pathlib import Path
from typing import List, Dict
from datetime import datetime

# Verificar e instalar openpyxl se necessário (para salvar Excel)
try:
    import openpyxl
except ImportError:
    print("📦 Instalando openpyxl...")
    import subprocess
    subprocess.check_call(["pip", "install", "openpyxl"])
    import openpyxl
    print("✅ openpyxl instalado")

# Configurações da Evolution API
EVOLUTION_URL = os.getenv("EVOLUTION_API_URL", "http://localhost:8080")
EVOLUTION_API_KEY = os.getenv("EVOLUTION_API_KEY")
INSTANCE = os.getenv("EVOLUTION_INSTANCE", "Revoluna")

if not EVOLUTION_API_KEY:
    raise ValueError("EVOLUTION_API_KEY não configurada. Configure no .env ou variáveis de ambiente.")

print(f"✅ Evolution API URL: {EVOLUTION_URL}")
print(f"✅ Instance: {INSTANCE}")
print(f"✅ API Key: {EVOLUTION_API_KEY[:10]}..." if len(EVOLUTION_API_KEY) > 10 else "✅ API Key configurada")


✅ Evolution API URL: http://localhost:8080
✅ Instance: Revoluna
✅ API Key: c24ecc525f...


In [ ]:
class EvolutionContactsCrawler:
    """Cliente para buscar contatos dos grupos via Evolution API."""
    
    def __init__(self):
        self.base_url = EVOLUTION_URL.rstrip("/")
        self.api_key = EVOLUTION_API_KEY
        self.instance = INSTANCE
        self.headers = {"apikey": self.api_key}
    
    async def testar_conexao(self) -> bool:
        """Testa se a conexão com a Evolution API está funcionando."""
        url = f"{self.base_url}/instance/fetchInstances"
        
        async with httpx.AsyncClient() as client:
            try:
                response = await client.get(url, headers=self.headers, timeout=10.0)
                response.raise_for_status()
                print(f"✅ Conexão com Evolution API OK (Status: {response.status_code})")
                return True
            except httpx.ConnectError as e:
                print(f"❌ Erro de conexão: Não foi possível conectar em {self.base_url}")
                print(f"   Verifique se a Evolution API está rodando e a URL está correta")
                return False
            except httpx.HTTPStatusError as e:
                print(f"❌ Erro HTTP {e.response.status_code}: {e.response.text[:200]}")
                return False
            except Exception as e:
                print(f"❌ Erro ao testar conexão: {type(e).__name__}: {e}")
                return False
    
    async def listar_grupos(self, com_participantes: bool = False, max_retries: int = 3) -> List[Dict]:
        """
        Lista todos os grupos que a instância participa.
        
        Args:
            com_participantes: Se True, retorna grupos com lista de participantes
            max_retries: Número máximo de tentativas em caso de timeout
        """
        # Seguindo o padrão do join_groups.py que está funcionando
        url = f"{self.base_url}/group/fetchAllGroups/{self.instance}?getParticipants={'true' if com_participantes else 'false'}"
        
        print(f"🔗 URL: {url}")
        print(f"🔑 Headers: apikey={self.api_key[:10]}...")
        
        # Timeout maior para muitos grupos (5 minutos)
        timeout = 300.0 if com_participantes else 180.0
        
        for tentativa in range(1, max_retries + 1):
            try:
                print(f"⏳ Tentativa {tentativa}/{max_retries} (timeout: {timeout}s)...")
                
                async with httpx.AsyncClient(timeout=timeout) as client:
                    response = await client.get(url, headers=self.headers)
                    print(f"📡 Status HTTP: {response.status_code}")
                    
                    response.raise_for_status()
                    grupos = response.json()
                    
                    # Verificar formato da resposta
                    if isinstance(grupos, dict):
                        # Pode retornar um objeto com 'groups' ou similar
                        if 'groups' in grupos:
                            grupos = grupos['groups']
                        elif 'data' in grupos:
                            grupos = grupos['data']
                        else:
                            print(f"⚠️ Resposta inesperada: {list(grupos.keys())}")
                            print(f"   Resposta completa: {grupos}")
                            return []
                    
                    if not isinstance(grupos, list):
                        print(f"⚠️ Resposta não é uma lista: {type(grupos)}")
                        print(f"   Conteúdo: {grupos}")
                        return []
                    
                    print(f"✅ Sucesso! {len(grupos)} grupos encontrados")
                    return grupos
                    
            except httpx.ReadTimeout as e:
                if tentativa < max_retries:
                    wait_time = tentativa * 10  # Backoff: 10s, 20s, 30s
                    print(f"⏱️ Timeout na tentativa {tentativa}. Aguardando {wait_time}s antes de tentar novamente...")
                    await asyncio.sleep(wait_time)
                    # Aumentar timeout na próxima tentativa
                    timeout = min(timeout * 1.5, 600.0)  # Máximo 10 minutos
                else:
                    print(f"❌ Timeout após {max_retries} tentativas")
                    print(f"   A requisição está demorando muito. Isso pode acontecer com muitos grupos.")
                    print(f"   💡 Tente novamente ou verifique se a Evolution API está processando a requisição")
                    return []
                    
            except httpx.ConnectError as e:
                print(f"❌ Erro de conexão: Não foi possível conectar em {self.base_url}")
                print(f"   Detalhes: {e}")
                return []
            except httpx.HTTPStatusError as e:
                print(f"❌ Erro HTTP {e.response.status_code}")
                print(f"   Resposta: {e.response.text[:500]}")
                if e.response.status_code == 404:
                    print(f"   💡 Verifique se a instância '{self.instance}' existe")
                elif e.response.status_code == 401:
                    print(f"   💡 Verifique se a API key está correta")
                return []
            except Exception as e:
                if tentativa < max_retries:
                    wait_time = tentativa * 5
                    print(f"⚠️ Erro na tentativa {tentativa}: {type(e).__name__}")
                    print(f"   Aguardando {wait_time}s antes de tentar novamente...")
                    await asyncio.sleep(wait_time)
                else:
                    print(f"❌ Erro ao listar grupos: {type(e).__name__}: {e}")
                    import traceback
                    print(f"   Traceback: {traceback.format_exc()}")
                    return []
        
        return []
    
    async def buscar_todos_participantes(self) -> Dict[str, List[Dict]]:
        """
        Busca todos os participantes de todos os grupos de uma vez.
        Retorna um dicionário: {grupo_jid: [participantes]}
        """
        grupos = await self.listar_grupos(com_participantes=True)
        
        resultado = {}
        for grupo in grupos:
            grupo_jid = grupo.get("id", "")
            participantes = grupo.get("participants", [])
            if grupo_jid:
                resultado[grupo_jid] = participantes
        
        return resultado

# Criar instância do crawler
crawler = EvolutionContactsCrawler()
print("✅ Crawler inicializado")

# Testar conexão primeiro
print("\n🔍 Testando conexão com Evolution API...")
conexao_ok = await crawler.testar_conexao()

# Verificar instâncias disponíveis
if conexao_ok:
    print("\n🔍 Verificando instâncias disponíveis...")
    url = f"{crawler.base_url}/instance/fetchInstances"
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(url, headers=crawler.headers, timeout=10.0)
            response.raise_for_status()
            instances = response.json()
            print(f"✅ Instâncias encontradas: {instances}")
            
            # Verificar se a instância configurada existe
            if isinstance(instances, list):
                # A API retorna 'name', não 'instanceName'
                instance_names = [inst.get('name', '') for inst in instances if isinstance(inst, dict)]
                print(f"   Instâncias disponíveis: {instance_names}")
                
                if INSTANCE not in instance_names:
                    print(f"\n⚠️ ATENÇÃO: A instância '{INSTANCE}' não foi encontrada!")
                    print(f"   Verifique a variável EVOLUTION_INSTANCE")
                else:
                    # Verificar status da conexão
                    instancia_info = next((inst for inst in instances if inst.get('name') == INSTANCE), None)
                    if instancia_info:
                        status = instancia_info.get('connectionStatus', 'unknown')
                        print(f"\n✅ Instância '{INSTANCE}' encontrada!")
                        print(f"   Status da conexão: {status}")
                        if status != 'open':
                            print(f"   ⚠️ A instância não está conectada (status: {status})")
            elif isinstance(instances, dict):
                print(f"   Formato de resposta: {list(instances.keys())}")
        except Exception as e:
            print(f"⚠️ Não foi possível verificar instâncias: {e}")


✅ Crawler inicializado

🔍 Testando conexão com Evolution API...
✅ Conexão com Evolution API OK (Status: 200)

🔍 Verificando instâncias disponíveis...
✅ Instâncias encontradas: [{'id': '49fe9268-dda3-44d9-8388-3e482cf81ef0', 'name': 'Revoluna', 'connectionStatus': 'open', 'ownerJid': '5511916175810@s.whatsapp.net', 'profileName': 'Julia', 'profilePicUrl': 'https://pps.whatsapp.net/v/t61.24694-24/534424589_1934969654116716_5298075044391826328_n.jpg?ccb=11-4&oh=01_Q5Aa3QGKEqW2pT6tvTPhKJ8A2_OElBVg8h2hXkmQriwyYnwhsg&oe=695FC118&_nc_sid=5e03e0&_nc_cat=104', 'integration': 'WHATSAPP-BAILEYS', 'number': None, 'businessId': None, 'token': 'EA3E1CF57A6D-49C6-89F7-21E74CE39269', 'clientName': 'evolution_exchange', 'disconnectionReasonCode': 401, 'disconnectionObject': '{"error":{"data":null,"isBoom":true,"isServer":false,"output":{"statusCode":401,"payload":{"statusCode":401,"error":"Unauthorized","message":"Log out instance: Revoluna"},"headers":{}}},"date":"2025-12-16T19:21:11.002Z"}', 'disconn

In [9]:
async def extrair_contatos_todos_grupos() -> List[Dict]:
    """Extrai todos os contatos de todos os grupos."""
    print("\n🔍 Buscando grupos (sem participantes primeiro)...")
    grupos = await crawler.listar_grupos(com_participantes=False)
    
    if not grupos:
        print("⚠️ Nenhum grupo encontrado!")
        print("💡 Possíveis causas:")
        print("   • A instância não está conectada ao WhatsApp")
        print("   • A instância não participa de nenhum grupo")
        print(f"   • Verifique o nome da instância (atual: {INSTANCE})")
        return []
    
    print(f"✅ Encontrados {len(grupos)} grupos")
    print("\n📋 Listando grupos:")
    for i, grupo in enumerate(grupos, 1):
        nome = grupo.get("subject", "Sem nome")
        jid = grupo.get("id", "")
        tamanho = grupo.get("size", 0)
        print(f"  {i}. {nome} ({tamanho} participantes) - {jid[:30]}...")
    
    print("\n🔍 Buscando participantes de todos os grupos...")
    print("⏳ Fazendo uma única requisição para buscar todos os participantes...\n")
    
    # Buscar todos os participantes de uma vez (mais eficiente)
    participantes_por_grupo = await crawler.buscar_todos_participantes()
    
    todos_contatos = []
    
    # Processar participantes de cada grupo
    for idx, grupo in enumerate(grupos, 1):
        grupo_jid = grupo.get("id", "")
        grupo_nome = grupo.get("subject", "Sem nome")
        participantes = participantes_por_grupo.get(grupo_jid, [])
        
        print(f"[{idx}/{len(grupos)}] Processando: {grupo_nome} ({len(participantes)} participantes)")
        
        for participante in participantes:
            # Extrair informações do participante
            participante_id = participante.get("id", "")
            phone_number = participante.get("phoneNumber", "")
            push_name = participante.get("pushName", "")
            name = participante.get("name", "")
            
            # Usar pushName se disponível, senão name, senão vazio
            nome_contato = push_name or name or ""
            
            # Extrair número de telefone
            telefone = ""
            if phone_number:
                # Formato geral: 5511999999999@s.whatsapp.net
                if "@s.whatsapp.net" in phone_number:
                    telefone = phone_number.split("@")[0]
                else:
                    telefone = phone_number
            elif participante_id and "@s.whatsapp.net" in participante_id:
                telefone = participante_id.split("@")[0]
            
            # Adicionar contato à lista
            todos_contatos.append({
                "Nome": nome_contato,
                "Numero_Telefone": telefone,
                "Grupo_Origem": grupo_nome,
                "Grupo_JID": grupo_jid
            })
    
    print(f"\n✅ Total de contatos coletados: {len(todos_contatos)}")
    return todos_contatos

# Executar extração
contatos = await extrair_contatos_todos_grupos()



🔍 Buscando grupos (sem participantes primeiro)...
🔗 URL: http://localhost:8080/group/fetchAllGroups/Revoluna?getParticipants=false
🔑 Headers: apikey=c24ecc525f...
❌ Erro ao listar grupos: ReadTimeout: 
   Traceback: Traceback (most recent call last):
  File "/Users/rafaelpivovar/Documents/Projetos/whatsapp-api/.venv/lib/python3.13/site-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    yield
  File "/Users/rafaelpivovar/Documents/Projetos/whatsapp-api/.venv/lib/python3.13/site-packages/httpx/_transports/default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rafaelpivovar/Documents/Projetos/whatsapp-api/.venv/lib/python3.13/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/Users/rafaelpivovar/Documents/Projetos/whatsapp-api/.venv/lib/python3.13/site-packages/httpcore/_async/co

In [ ]:
# Criar DataFrame
df = pd.DataFrame(contatos)

# Remover duplicatas (mesmo número em grupos diferentes)
print(f"📊 Total de registros: {len(df)}")
print(f"📊 Contatos únicos (por número): {df['Numero_Telefone'].nunique()}")

# Mostrar estatísticas
print("\n📈 Estatísticas:")
print(f"  • Grupos processados: {df['Grupo_Origem'].nunique()}")
print(f"  • Contatos com nome: {df[df['Nome'] != '']['Nome'].count()}")
print(f"  • Contatos sem nome: {df[df['Nome'] == '']['Nome'].count()}")

# Mostrar preview
print("\n👀 Preview dos dados:")
print(df.head(10))


In [ ]:
# Criar diretório de saída se não existir
output_dir = Path("data/contatos_grupos")
output_dir.mkdir(parents=True, exist_ok=True)

# Gerar nome do arquivo com timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
csv_file = output_dir / f"contatos_grupos_{timestamp}.csv"
excel_file = output_dir / f"contatos_grupos_{timestamp}.xlsx"

# Salvar CSV
df.to_csv(csv_file, index=False, encoding='utf-8-sig')
print(f"✅ CSV salvo: {csv_file}")

# Salvar Excel
df.to_excel(excel_file, index=False, engine='openpyxl')
print(f"✅ Excel salvo: {excel_file}")

print(f"\n📁 Arquivos salvos em: {output_dir.absolute()}")
print(f"   • {csv_file.name}")
print(f"   • {excel_file.name}")


In [ ]:
# Análise adicional: contatos por grupo
print("📊 Contatos por grupo:")
contatos_por_grupo = df.groupby('Grupo_Origem').size().sort_values(ascending=False)
print(contatos_por_grupo)

# Contatos que aparecem em múltiplos grupos
print("\n👥 Contatos em múltiplos grupos:")
contatos_multiplos = df.groupby('Numero_Telefone').agg({
    'Nome': 'first',
    'Grupo_Origem': lambda x: ', '.join(x.unique()),
    'Numero_Telefone': 'count'
}).rename(columns={'Numero_Telefone': 'Qtd_Grupos'})
contatos_multiplos = contatos_multiplos[contatos_multiplos['Qtd_Grupos'] > 1].sort_values('Qtd_Grupos', ascending=False)
print(f"Total: {len(contatos_multiplos)} contatos aparecem em mais de um grupo")
print(contatos_multiplos.head(20))
